# Break Captchas Into Separate Characters

Import Libraries

In [ ]:
import cv2                                  # used for image processing
import numpy as np                          # used to store image data in numpy arrays
import os                                   # used to read files from directory
import matplotlib.pyplot as plt             # used to display images

In [ ]:
# Dataset : https://www.kaggle.com/datasets/greysky/captcha-dataset
DIR = os.getcwd() + '/samples/'            # path to the directory containing the images
DATASET = os.getcwd() + '/dataset/'         # path to the directory where characters will be stored

RESIZED_IMAGE_WIDTH = 20                    # width of resized image
RESIZED_IMAGE_HEIGHT = 30                   # height of resized image

In [ ]:
try:
    os.mkdir(DATASET)                       # create directory to store characters
except:
    pass

Function for Image Preprocessing

In [ ]:
def preprocess(img):
    # convert to grayscale
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # get binary image
    thresh = cv2.adaptiveThreshold( imgGray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 0)

    # remove noise from image
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))
    
    # get dilated image
    dilate = cv2.dilate(close, np.ones((2, 2), np.uint8), iterations=1)

    # invert image colors
    image = cv2.bitwise_not(dilate)

    # display image
    plt.imshow(image, cmap='gray')
    plt.show()

    return image

Break Captchas

In [ ]:
for img in os.listdir(DIR):
    image = cv2.imread(DIR + img)           # read captcha image
    imgname = img.split('.')[0]             # get captcha text
    print(imgname)
    image = preprocess(image)               # preprocess image

    IMAGE_WIDTH = image.shape[1]            # get image width
    IMAGE_HEIGHT = image.shape[0]           # get image height

    # get separate alphabets from captcha
    for i in range(0, IMAGE_WIDTH, IMAGE_WIDTH//5):
        # divide image into 5 parts
        print(img[int(i/(IMAGE_WIDTH//5))])
        # get each part
        letter = image[5:IMAGE_HEIGHT-5, i:i+IMAGE_WIDTH//5]
        # resize each part to 20x30
        letter = cv2.resize(letter, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
        # dilate each part
        letter = cv2.dilate(letter, np.ones((2, 2), np.uint8), iterations=1)

        plt.imshow(letter, cmap='gray')
        plt.show()
        
        # save each part
        PATH = DATASET + img[int(i/(IMAGE_WIDTH//5))] + '/'
        try:
            os.chdir(PATH)
        except FileNotFoundError:
            os.mkdir(PATH)
        finally:
            dirlen = len(os.listdir(PATH))
        cv2.imwrite(PATH + str(dirlen) + '.png', letter)
    cv2.destroyAllWindows()                # close all windows
    cv2.waitKey(1)